In [ ]:
import numpy as np
import xarray as xr
import pickle
import pandas as pd
import time
import os

import datetime
from sys import getsizeof,path


import matplotlib.pyplot as plt
import matplotlib

import theano
import theano.tensor as tt
import pymc3 as pm
theano.config.optimizer="fast_run"

path.append("../../src")
path.append("../../Data/Bundeslaender/BW")

from BW_Deaths_ManualParser import BW_ReportedDeaths

In [ ]:
deaths = BW_ReportedDeaths()

In [ ]:
print(deaths.coords)
print(deaths.shape)

In [ ]:
#plt.bar(deaths.coords["date"],deaths.sel(publication=datetime.datetime(2021,1,3)))
print(deaths.coords)

In [ ]:
fig = plt.figure(figsize=(14,8))
gs = fig.add_gridspec(1,1)

ax = fig.add_subplot(gs[0,0])

z = deaths.isel(publication=0)
first = deaths.coords["date"].values[0]
color = plt.cm.plasma(np.array([range(7)])/6.)[0]

for ipub,pub in enumerate(deaths.coords["publication"][1:]):
    p = pub.values
    n = ((p-first) / np.timedelta64(1, 'D')).astype("int")+10
    dt = pd.to_datetime(p.astype("int"))
    
    d = deaths.sel(publication=pub) - z
    ax.bar(deaths.coords["date"][:n],d[:n],bottom=z[:n],label=p,color=color[dt.weekday()])
    z += d
    
print()   
#ax.legend()
plt.savefig("BW_deaths4.pdf")

In [ ]:
dd = deaths.diff("publication",label="upper").sel(date=datetime.datetime(2020,12,20))
d = deaths.sel(date=datetime.datetime(2020,12,20))

print(dd.shape,d.shape)
print(dd)
print(d)


In [ ]:
d = deaths.sel(publication=datetime.datetime(2020,3,11))
print(d.shape)
print(d)

In [ ]:
sdeaths = deaths.sel(date=slice(datetime.datetime(2020,1,1),datetime.datetime(2021,3,1)))

z = np.zeros((7,60,),dtype="int32")

end = sdeaths.coords["publication"][-1].values
print(end)
for dt in sdeaths.coords["date"]:
    dtv=dt.values
    dtw = dt.dt.dayofweek.values
    dz = sdeaths.sel(date=dtv,publication=slice(dtv,end)).values[:60] #.diff("publication")
    z[dtw] += dz
    

zn = z/z[:,-1:]
for day in range(7):
    plt.plot(range(day,day+len(zn[day])),zn[day])

print(z.shape)

tr = [.1,.2,1/3.,.5,.8,.9]
print(tr)
t = np.zeros((7,len(tr)),dtype="int32")
for i in range(60):
    for j in range(7):
        for k,r in enumerate(tr):
            if zn[j,i] < r:
                t[j,k] = i
        
print(t)

In [ ]:
for day in range(7):
    znd = np.diff(zn[day])
    
    plt.plot(range(day,day+len(znd)),np.diff(zn[day]))


In [ ]:
plt.plot(np.transpose(z))